In [5]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 1
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-47.5695707105
crossvalidation 1 compeleted
cost time   3.2 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-46.8335426086
crossvalidation 2 compeleted
cost time   5.9 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-48.2013024608
crossvalidation 3 compeleted
cost time   9.0 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mbwm0 False: mklt0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: fjmg0 False: fadg0
error ! True: mgrt0 False: mtmr0
done
-47.9894747121
crossvalidation 4 compeleted
cost time  11.9 minute
the crossval 3 detection_rate is 0.9823529411764705
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
-48.8556305522
crossvalidation 5 compeleted
cost time  15.0 minute
the crossval 4 detection_rate 

In [7]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 5
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=5,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.7027540488
crossvalidation 1 compeleted
cost time   4.2 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-51.3544794329
crossvalidation 2 compeleted
cost time   7.9 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.7327408861
crossvalidation 3 compeleted
cost time  12.1 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mbwm0 False: mklt0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: fjmg0 False: fadg0
done
-53.2836296974
crossvalidation 4 compeleted
cost time  15.9 minute
the crossval 3 detection_rate is 0.9882352941176471
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
-53.2793045139
crossvalidation 5 compeleted
cost time  20.1 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalida

In [9]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 10
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=10,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.5786696427
crossvalidation 1 compeleted
cost time   6.6 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
error ! True: mrtk0 False: mcem0>>>>>>>>>>>>>>>>>>>]100.00%
done
-51.4374551961
crossvalidation 2 compeleted
cost time  12.6 minute
the crossval 1 detection_rate is 0.9941176470588236
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.6102440671
crossvalidation 3 compeleted
cost time  18.8 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: fjmg0 False: fadg0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: mkcl0 False: mtmr0
error ! True: mgrt0 False: mdls0
done
-53.1993123302
crossvalidation 4 compeleted
cost time  23.8 minute
the crossval 3 detection_rate is 0.9823529411764705
crossvalidation 5 start
error ! True: fcft0 False: fmah0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: mrrk0 False: mcrc0
done
-53.1547827407
crossvalidation 5 compeleted


In [11]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 15
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=15,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.6886082543
crossvalidation 1 compeleted
cost time   7.1 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
error ! True: mrtk0 False: mcem0>>>>>>>>>>>>>>>>>>>]100.00%
done
-51.581866771
crossvalidation 2 compeleted
cost time  13.4 minute
the crossval 1 detection_rate is 0.9941176470588236
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.6535178868
crossvalidation 3 compeleted
cost time  19.9 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mjdh0 False: mhpg0>>>>>>>>>>>>>>>>>>>]100.00%
done
-53.2707222762
crossvalidation 4 compeleted
cost time  26.2 minute
the crossval 3 detection_rate is 0.9941176470588236
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
-53.1984275645
crossvalidation 5 compeleted
cost time  32.6 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalidation 6 start
d

In [ ]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 20
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=20,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.6628346729
crossvalidation 1 compeleted
cost time   8.2 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
error ! True: mrtk0 False: mcem0>>>>>>>>>>>>>>>>>>>]100.00%
done
-51.566496504
crossvalidation 2 compeleted
cost time  16.2 minute
the crossval 1 detection_rate is 0.9941176470588236
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.599005983
crossvalidation 3 compeleted
cost time  24.1 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mjdh0 False: mhpg0>>>>>>>>>>>>>>>>>>>]100.00%
done
-53.2251439152
crossvalidation 4 compeleted
cost time  32.6 minute
the crossval 3 detection_rate is 0.9941176470588236
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
-53.1556042326
crossvalidation 5 compeleted
cost time  42.0 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalidation 6 start
do

In [ ]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 25
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=25,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.6291740609
crossvalidation 1 compeleted
cost time   9.7 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-51.5062548128
crossvalidation 2 compeleted
cost time  19.6 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.5165567075
crossvalidation 3 compeleted
cost time  28.8 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
-53.1490297128
crossvalidation 4 compeleted
cost time  37.0 minute
the crossval 3 detection_rate is 1.0
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
-53.0859752657
crossvalidation 5 compeleted
cost time  45.2 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalidation 6 start


In [ ]:
'''
tuning the convergence 1,5,10,15,20,25,30
'''

'''
convergence 30
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
scores_sum=0
for cross_num in range(10):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples,num_samples))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=30,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

    '''
    Calculate the detection rate
    '''
    for index in range(num_samples):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
            confusion_matrix[index,index] +=1
        else:
            false_num +=1
            false_sum +=1
            confusion_matrix[index,test_index] +=1
            print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()
    scores_sum +=np.sum(scores_set)/(num_samples**2)
    print(np.sum(scores_set)/(num_samples**2))
    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("overall expected scores",scores_sum/10)

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()


